In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from ib_insync import *
from finta import TA  
from datetime import datetime, timedelta

In [2]:
util.startLoop()

In [3]:
# Conexión a IBKR
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

<IB connected to 127.0.0.1:7497 clientId=1>

In [4]:
# Configuración de predicción
PREDICTION_HOUR = "17:35"
NUM_CANDLES = 8  # 10 velas de predicción (50 min en total)

In [5]:
# Definir el contrato de GBP/USD
contract = Forex('GBPUSD')

In [6]:
# Establecer el rango de tiempo
end_time = datetime.now()
start_time = end_time - timedelta(days=30)

# Obtener datos históricos
bars = ib.reqHistoricalData(
    contract,
    endDateTime=end_time.strftime('%Y%m%d %H:%M:%S'),
    durationStr='30 D',
    barSizeSetting='5 mins',
    whatToShow='MIDPOINT',
    useRTH=False
)

In [7]:
# Convertir los datos a DataFrame
df = util.df(bars)
df['datetime'] = pd.to_datetime(df['date'])
df.set_index('datetime', inplace=True)
df = df[['open', 'high', 'low', 'close', 'volume']]

In [8]:
# Cálculo de indicadores técnicos con finta
df['rsi'] = TA.RSI(df)
df['macd'] = TA.MACD(df)['MACD']
df['macd_signal'] = TA.MACD(df)['SIGNAL']
df['sma50'] = TA.SMA(df, 50)
df['sma200'] = TA.SMA(df, 200)
df.dropna(inplace=True)

In [9]:
# Filtrar las columnas antes de normalizar
df_features = ['open', 'high', 'low', 'close', 'rsi', 'macd', 'macd_signal', 'sma50', 'sma200']
df = df[df_features]


In [10]:
# Normalización de datos
df_min = df.min()
df_max = df.max()
df = (df - df_min) / (df_max - df_min)

In [11]:
# Preprocesamiento de datos para LSTM
data = df.values
X, y = [], []
for i in range(len(data) - NUM_CANDLES):
    X.append(data[i:i+NUM_CANDLES])
    y.append(data[i+NUM_CANDLES][3])  # Predicción basada en 'close'
X, y = np.array(X), np.array(y)


In [12]:
# Construcción del modelo LSTM
model = Sequential([
    Input(shape=(X.shape[1], X.shape[2])),
    LSTM(128, return_sequences=True, activation='tanh'),
    Dropout(0.2),
    LSTM(128, return_sequences=True, activation='tanh'),
    Dropout(0.2),
    LSTM(64, activation='tanh'),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mse')

# Entrenar el modelo
model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0169 - val_loss: 0.0044
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0010 - val_loss: 0.0059
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 6.9881e-04 - val_loss: 0.0053
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 6.1157e-04 - val_loss: 0.0062
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 4.8852e-04 - val_loss: 0.0119
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 4.6250e-04 - val_loss: 0.0071
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 3.8875e-04 - val_loss: 0.0045
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 3.3317e-04 - val_loss: 0.0063
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 2.7564e-04 - val_loss: 0.0025
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 2.5128e-04 - val_loss: 0.0035
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 2.4448e-04 - val_loss: 0.0035
Epoch

In [15]:
# Realizar predicciones
last_sequence = np.array([data[-NUM_CANDLES:]])  # Última secuencia para predecir
predictions = model.predict(last_sequence)  # Asegurar salida correcta
predictions = predictions.flatten()  # Convertir a 1D

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


In [16]:
# Asegurar que predictions tiene NUM_CANDLES elementos
if len(predictions) != NUM_CANDLES:
    predictions = np.resize(predictions, NUM_CANDLES)  # Ajustar tamaño si es necesario


In [17]:
# Desnormalizar predicciones
predictions = predictions * (df_max['close'] - df_min['close']) + df_min['close']

# Crear DataFrame con predicciones
prediction_times = pd.date_range(start=df.index[-1], periods=NUM_CANDLES, freq='5T')
prediction_df = pd.DataFrame({'datetime': prediction_times, 'predicted_close': predictions})

# Mostrar resultados
print(prediction_df)

                   datetime  predicted_close
0 2025-03-10 17:30:00-05:00          1.28579
1 2025-03-10 17:35:00-05:00          1.28579
2 2025-03-10 17:40:00-05:00          1.28579
3 2025-03-10 17:45:00-05:00          1.28579
4 2025-03-10 17:50:00-05:00          1.28579
5 2025-03-10 17:55:00-05:00          1.28579
6 2025-03-10 18:00:00-05:00          1.28579
7 2025-03-10 18:05:00-05:00          1.28579


C:\Users\alex_\AppData\Local\Temp\ipykernel_4580\3203699912.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  prediction_times = pd.date_range(start=df.index[-1], periods=NUM_CANDLES, freq='5T')
